In [1]:
import os
import librosa
from librosa import display
import numpy as np
from matplotlib import pyplot as plt
import time
import random

/usr/lib/python3/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Listing all samples

In [2]:
DATASET_PATH = "samples"
samples = os.listdir(DATASET_PATH)
random.shuffle(samples)

# EmotionsDict = {'N': 'Neutral', 'F':'Fear', 'H':'Happiness', 'S':'Sadness', 'W':'Surprise', 'A':'Angry'}
EmotionsDict = {'N': 0, 'F': 1, 'H': 2, 'S': 3, 'W': 4, 'A': 5}
emotions = [EmotionsDict[x[3]] for x in samples]

# Preprocessing data and creating features

In [3]:
sr = 44100

SIG_LENGTH = 200

mfccs = []

for s in samples:
    t1 = time.time()

    sig, sr = librosa.load(f'{DATASET_PATH}/{s}', sr=sr)
    sig, _ = librosa.effects.trim(sig) # trimming beginning and ending silence


    # MFCC config
    WinLen = int(0.040 * sr) # 40 milisecond
    WinHop = WinLen // 2
    
    sig_mfcc = librosa.feature.mfcc(y=librosa.power_to_db(sig), sr=sr, n_mfcc=12, fmax=sr//2)

    # cutting every longer sequence than SIG_LENGTH and padding any sequence shorter than SIG_LENGTH
    if sig_mfcc.shape[1] < SIG_LENGTH:
        sig_mfcc = np.pad(sig_mfcc, [(0, 0), (0, SIG_LENGTH - sig_mfcc.shape[1])], mode='symmetric')
    elif sig_mfcc.shape[1] > SIG_LENGTH:
        sig_mfcc = sig_mfcc[:, :SIG_LENGTH]

    mfccs.append(sig_mfcc)

In [1]:
mfccs = np.array(mfccs)
emotions = np.array(emotions)

print(mfccs.shape)
print(emotions.shape)

NameError: name 'np' is not defined

In [5]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import datetime
from matplotlib import pyplot as plt
from pretty_confusion_matrix import pp_matrix_from_data

2023-01-16 21:04:56.925052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 21:04:57.228695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 21:04:57.228746: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-16 21:04:59.430911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [6]:
BATCH_SIZE = 64
LAYERS_ACTIVATION = 'relu'
LOSS_FUNCTION = 'sparse_categorical_crossentropy'

EPOCHS = 20

def get_model():
    model = tf.keras.Sequential([
              tf.keras.layers.Conv2D(64, (3, 3), activation=LAYERS_ACTIVATION, padding='same'),
              tf.keras.layers.Conv2D(64, (3, 3), activation=LAYERS_ACTIVATION, padding='same'),
              tf.keras.layers.MaxPool2D((2, 2)),
              tf.keras.layers.Conv2D(128, (3, 3), activation=LAYERS_ACTIVATION, padding='same'),
              tf.keras.layers.Conv2D(128, (3, 3), activation=LAYERS_ACTIVATION, padding='same'),
              tf.keras.layers.MaxPool2D((2, 2)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(128, activation=LAYERS_ACTIVATION),
              tf.keras.layers.Dense(64, activation=LAYERS_ACTIVATION),
              tf.keras.layers.Dense(6, activation=tf.keras.activations.softmax)               
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=LOSS_FUNCTION, metrics=['accuracy'])
    return model

model = get_model()
history = model.fit(mfccs, emotions, epochs=EPOCHS, batch_size=BATCH_SIZE)

: 

: 